In [1]:
import os
import pandas as pd
import numpy as np
import time, gc
import cv2
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pretrainedmodels
from argparse import Namespace
from sklearn.utils import shuffle
from apex import amp
from sklearn.model_selection import StratifiedKFold
from efficientnet_pytorch import EfficientNet
from cvcore.data.auto_augment import RandAugment
from PIL import Image
from utils import bn_update, moving_average, copy_model


In [2]:
!ls /home/chec/data/bengali

class_map.csv		       train.csv
sample_submission.csv	       train.csv.zip
test.csv		       train_image_data_0.parquet
test_image_data_0.parquet      train_image_data_0.parquet.zip
test_image_data_0.parquet.zip  train_image_data_1.parquet
test_image_data_1.parquet      train_image_data_1.parquet.zip
test_image_data_1.parquet.zip  train_image_data_2.parquet
test_image_data_2.parquet      train_image_data_2.parquet.zip
test_image_data_2.parquet.zip  train_image_data_3.parquet
test_image_data_3.parquet      train_image_data_3.parquet.zip
test_image_data_3.parquet.zip


In [3]:
#!ls /home/chec/data/bengali

In [4]:
DATA_DIR = '/home/chec/data/bengali'

In [5]:
train_df = pd.read_csv(f'{DATA_DIR}/train.csv')
test_df = pd.read_csv(f'{DATA_DIR}/test.csv')
class_map_df = pd.read_csv(f'{DATA_DIR}/class_map.csv')
sample_sub_df = pd.read_csv(f'{DATA_DIR}/sample_submission.csv')

In [6]:
train_df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [7]:
HEIGHT = 137
WIDTH = 236

In [8]:
#import albumentations as albu
def get_train_augs():
    return RandAugment(n=2, m=27)

In [9]:
#plt.imshow(x)

In [10]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms


class BengaliDataset(Dataset):
    def __init__(self, df, img_df, train_mode=True, test_mode=False):
        self.df = df
        self.img_df = img_df
        self.train_mode = train_mode
        self.test_mode = test_mode

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = self.get_img(row.image_id)
        orig_img = img.copy()
        #print(img.shape)
        if self.train_mode:
            augs = get_train_augs()
            #img = augs(image=img)['image']
            img = np.asarray(augs(Image.fromarray(img)))
        
        img = np.expand_dims(img, axis=-1)
        orig_img = np.expand_dims(orig_img, axis=-1)
        
        #print('###', img.shape)
        #img = np.concatenate([img, img, img], 2)
        #print('>>>', img.shape)
        
        # taken from https://www.kaggle.com/iafoss/image-preprocessing-128x128
        #MEAN = [ 0.06922848809290576,  0.06922848809290576,  0.06922848809290576]
        #STD = [ 0.20515700083327537,  0.20515700083327537,  0.20515700083327537]
        
        img = transforms.functional.to_tensor(img)
        orig_img = transforms.functional.to_tensor(orig_img)
        
        #img = transforms.functional.normalize(img, mean=MEAN, std=STD)
        
        if self.test_mode:
            return img
        elif self.train_mode:
            return img, orig_img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
        else:
            return img, torch.tensor([row.grapheme_root, row.vowel_diacritic, row.consonant_diacritic, row.word_label])
                    
    def get_img(self, img_id):
        return 255 - self.img_df.loc[img_id].values.reshape(HEIGHT, WIDTH).astype(np.uint8)

    def __len__(self):
        return len(self.df)
    
def get_train_val_loaders(batch_size=4, val_batch_size=4, ifold=0, dev_mode=False):
    train_df = pd.read_csv(f'{DATA_DIR}/train.csv')

    train_df = shuffle(train_df, random_state=1234)

    grapheme_words = np.unique(train_df.grapheme.values)
    grapheme_words_dict = {grapheme: i for i, grapheme in enumerate(grapheme_words)}
    train_df['word_label'] = train_df['grapheme'].map(lambda x: grapheme_words_dict[x])

    print(train_df.shape)

    if dev_mode:
        img_df = pd.read_parquet(f'{DATA_DIR}/train_image_data_0.parquet').set_index('image_id')
        train_df = train_df.iloc[:1000]
    else:
        img_dfs = [pd.read_parquet(f'{DATA_DIR}/train_image_data_{i}.parquet') for i in range(4)]
        img_df = pd.concat(img_dfs, axis=0).set_index('image_id')
    print(img_df.shape)
    #split_index = int(len(train_df) * 0.9)
    
    #train = train_df.iloc[:split_index]
    #val = train_df.iloc[split_index:]
    
    kf = StratifiedKFold(5, random_state=1234, shuffle=True)
    for i, (train_idx, val_idx) in enumerate(kf.split(train_df, train_df['grapheme_root'].values)):
        if i == ifold:
            #print(val_idx)
            train = train_df.iloc[train_idx]
            val = train_df.iloc[val_idx]
            break
    assert i == ifold
    print(train.shape, val.shape)
    
    train_ds = BengaliDataset(train, img_df, True, False)
    val_ds = BengaliDataset(val, img_df, False, False)
    
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=8, drop_last=True)
    train_loader.num = len(train_ds)

    val_loader = DataLoader(val_ds, batch_size=val_batch_size, shuffle=False, num_workers=8, drop_last=False)
    val_loader.num = len(val_ds)

    return train_loader, val_loader

In [11]:
#train_loader, val_loader = get_train_val_loaders()

In [12]:
#for x in train_loader:
#    print(x)
#    break

# model

In [13]:
#import pretrainedmodels

In [14]:
#model_name = 'resnet50' # could be fbresnet152 or inceptionresnetv2
#model = pretrainedmodels.__dict__[model_name](num_classes=1000, pretrained='imagenet').cuda()
#model.eval()

In [15]:
from argparse import Namespace
import timm
from timm.models.activations import Swish, Mish
from timm.models.adaptive_avgmax_pool import SelectAdaptivePool2d

MEAN = [ 0.06922848809290576 ]
STD = [ 0.20515700083327537 ]

class BengaliNet(nn.Module):
    """
    EfficientNet B0-B8.
    Args:
        cfg (CfgNode): configs
    """
    def __init__(self, cfg):
        super(BengaliNet, self).__init__()
        model_name = cfg.MODEL_NAME
        pretrained = cfg.PRETRAINED
        input_channels = cfg.IN_CHANNELS
        pool_type = cfg.POOL_TYPE
        drop_connect_rate = cfg.DROP_CONNECT
        self.drop_rate = cfg.DROPOUT
        cls_head = cfg.CLS_HEAD
        num_total_classes = cfg.NUM_GRAPHEME_CLASSES + cfg.NUM_VOWEL_CLASSES + cfg.NUM_CONSONANT_CLASSES \
            + cfg.NUM_WORD_CLASSES

        backbone = timm.create_model(
            model_name=model_name,
            pretrained=pretrained,
            in_chans=input_channels,
            drop_connect_rate=drop_connect_rate,
        )
        self.conv_stem = backbone.conv_stem
        self.bn1 = backbone.bn1
        self.act1 = backbone.act1
        ### Original blocks ###
        for i in range(len((backbone.blocks))):
            setattr(self, "block{}".format(str(i)), backbone.blocks[i])
        self.conv_head = backbone.conv_head
        self.bn2 = backbone.bn2
        self.act2 = backbone.act2
        self.aux_block5 = backbone.blocks[5]
        self.aux_num_features = self.block5[-1].bn3.num_features
        self.aux_head4 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn4 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act4 = Swish()
        self.aux_head5 = nn.Conv2d(self.aux_num_features, self.aux_num_features * 4, kernel_size=(1, 1), stride=(1, 1), bias=False)
        self.bn5 = nn.BatchNorm2d(self.aux_num_features * 4)
        self.act5 = Swish()
        self.global_pool = SelectAdaptivePool2d(pool_type=pool_type)
        self.num_features = backbone.num_features * self.global_pool.feat_mult()
        assert cls_head == 'linear'
        if cls_head == "linear":
            ### Baseline head ###
            self.fc = nn.Linear(self.num_features, num_total_classes)            
            self.aux_fc1 = nn.Linear(self.aux_num_features*4, num_total_classes)
            self.aux_fc2 = nn.Linear(self.aux_num_features*4, num_total_classes)
            
            for fc in [self.fc, self.aux_fc1, self.aux_fc2]:
                nn.init.zeros_(fc.bias.data)
        elif cls_head == "norm_softmax":
            ### NormSoftmax ###
            self.grapheme_fc = NormSoftmax(self.num_features, num_grapheme_classes)
            self.consonant_fc = NormSoftmax(self.num_features, num_consonant_classes)
            self.vowel_fc = NormSoftmax(self.num_features, num_vowel_classes)
        # Replace with Mish activation
        if cfg.MODEL_ACTIVATION == "mish":
            convert_swish_to_mish(self)
        del backbone

    def _features(self, x):
        x = self.conv_stem(x)
        x = self.bn1(x)
        x = self.act1(x)
        x = self.block0(x)
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x); b4 = x
        x = self.block5(x); b4 = self.aux_block5(b4); b5 = x
        x = self.block6(x)
        x = self.conv_head(x); b4 = self.aux_head4(b4); b5 = self.aux_head5(b5)
        x = self.bn2(x); b4 = self.bn4(b4); b5 = self.bn5(b5)
        x = self.act2(x); b4 = self.act4(b4); b5 = self.act5(b5)
        return b4, b5, x

    def forward(self, x):
        for i in range(len(x)):
            transforms.functional.normalize(x[i], mean=MEAN, std=STD, inplace=True)

        # _, _, x = self._features(x)
        b4, b5, x = self._features(x)
        x = self.global_pool(x); b4 = self.global_pool(b4); b5 = self.global_pool(b5)
        x = torch.flatten(x, 1); b4 = torch.flatten(b4, 1); b5 = torch.flatten(b5, 1)
        if self.drop_rate > 0.:
            x = F.dropout(x, p=self.drop_rate, training=self.training)
        logits = self.fc(x)
        
        aux_logits1 = self.aux_fc1(b4)
        aux_logits2 = self.aux_fc2(b5)
        
        return logits, aux_logits1, aux_logits2

In [16]:
MODEL_DIR = './model4-ckps'
def create_model(cfg):
    model = BengaliNet(cfg)
    model_file = os.path.join(MODEL_DIR, cfg.MODEL_NAME, cfg.CKP_NAME)

    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)

    print('model file: {}, exist: {}'.format(model_file, os.path.exists(model_file)))

    if os.path.exists(model_file):
        print('loading {}...'.format(model_file))
        model.load_state_dict(torch.load(model_file))
    
    return model, model_file

In [17]:
#bnet = BengaliNet('se_resnext50_32x4d').cuda()

In [18]:
#bnet(torch.randn((2, 1, 137, 236)).cuda()).size()

# train

In [19]:
round(1/9, 6)

0.111111

In [20]:
import numpy as np
import sklearn.metrics
import torch


def calc_metrics(preds0, preds1, preds2, preds3, y):
    assert len(y) == len(preds0) == len(preds1) == len(preds2) == len(preds3)

    recall_grapheme = sklearn.metrics.recall_score(preds0, y[:, 0], average='macro')
    recall_vowel = sklearn.metrics.recall_score(preds1, y[:, 1], average='macro')
    recall_consonant = sklearn.metrics.recall_score(preds2, y[:, 2], average='macro')
    recall_word = sklearn.metrics.recall_score(preds3, y[:, 3], average='macro')
    
    scores = [recall_grapheme, recall_vowel, recall_consonant]
    final_recall_score = np.average(scores, weights=[2, 1, 1])
    
    metrics = {}
    metrics['recall'] = round(final_recall_score, 6)
    metrics['recall_grapheme'] = round(recall_grapheme, 6)
    metrics['recall_vowel'] = round(recall_vowel, 6)
    metrics['recall_consonant'] = round(recall_consonant, 6)
    metrics['recall_word'] = round(recall_word, 6)
    
    metrics['acc_grapheme'] = round((preds0 == y[:, 0]).sum() / len(y), 6)
    metrics['acc_vowel'] = round((preds1 == y[:, 1]).sum() / len(y), 6)
    metrics['acc_consonant'] = round((preds2 == y[:, 2]).sum() / len(y), 6)
    metrics['acc_word'] = round((preds3 == y[:, 3]).sum() / len(y), 6)    
    
    return metrics

In [21]:
def criterion(outputs, y_true):
    # outputs: (N, 182)
    # y_true: (N, 3)
    
    outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
    loss0 = F.cross_entropy(outputs[0], y_true[:, 0], reduction='mean')
    loss1 = F.cross_entropy(outputs[1], y_true[:, 1], reduction='mean')
    loss2 = F.cross_entropy(outputs[2], y_true[:, 2], reduction='mean')
    loss3 = F.cross_entropy(outputs[3], y_true[:, 3], reduction='mean')
    
    return loss0 + loss1 + loss2 + loss3 #, loss0.item(), loss1.item(), loss2.item()

In [22]:
def validate(model, val_loader):
    model.eval()
    loss0, loss1, loss2, loss3 = 0., 0., 0., 0.
    preds0, preds1, preds2, preds3 = [], [], [], []
    y_true = []
    with torch.no_grad():
        for x, y in val_loader:
            y_true.append(y)
            x, y = x.cuda(), y.cuda()
            outputs, _, _ = model(x)
            outputs = torch.split(outputs, [168, 11, 7, 1295], dim=1)
            
            preds0.append(torch.max(outputs[0], dim=1)[1])
            preds1.append(torch.max(outputs[1], dim=1)[1])
            preds2.append(torch.max(outputs[2], dim=1)[1])
            preds3.append(torch.max(outputs[3], dim=1)[1])
            loss0 += F.cross_entropy(outputs[0], y[:, 0], reduction='sum').item()
            loss1 += F.cross_entropy(outputs[1], y[:, 1], reduction='sum').item()
            loss2 += F.cross_entropy(outputs[2], y[:, 2], reduction='sum').item()
            loss3 += F.cross_entropy(outputs[3], y[:, 3], reduction='sum').item()
            
            # for debug
            #metrics = {}
            #metrics['loss_grapheme'] =  F.cross_entropy(outputs[0], y[:, 0], reduction='mean').item()
            #metrics['loss_vowel'] =  F.cross_entropy(outputs[1], y[:, 1], reduction='mean').item()
            #metrics['loss_consonant'] =  F.cross_entropy(outputs[2], y[:, 2], reduction='mean').item()
            #return metrics
    
    preds0 = torch.cat(preds0, 0).cpu().numpy()
    preds1 = torch.cat(preds1, 0).cpu().numpy()
    preds2 = torch.cat(preds2, 0).cpu().numpy()
    preds3 = torch.cat(preds3, 0).cpu().numpy()
    
    y_true = torch.cat(y_true, 0).numpy()
    
    #print('y_true:', y_true.shape)
    #print('preds0:', preds0.shape)
    
    metrics = calc_metrics(preds0, preds1, preds2, preds3, y_true)
    metrics['loss_grapheme'] = round(loss0 / val_loader.num, 6)
    metrics['loss_vowel'] = round(loss1 / val_loader.num, 6)
    metrics['loss_consonant'] = round(loss2 / val_loader.num, 6)
    metrics['loss_word'] = round(loss3 / val_loader.num, 6)
    
    return metrics
            

In [23]:
def get_lrs(optimizer):
    lrs = []
    for pgs in optimizer.state_dict()['param_groups']:
        lrs.append(pgs['lr'])
    lrs = ['{:.6f}'.format(x) for x in lrs]
    return lrs

In [24]:
def save_model(model, model_file):
    parent_dir = os.path.dirname(model_file)
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
    if isinstance(model, nn.DataParallel):
        torch.save(model.module.state_dict(), model_file)
    else:
        torch.save(model.state_dict(), model_file)

In [25]:
def mixup(data, targets, alpha=1):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)
    data = data * lam + shuffled_data * (1 - lam)
    targets = (targets, shuffled_targets, lam)

    return data, targets


def mixup_criterion(outputs, targets):
    targets1, targets2, lam = targets
    #criterion = nn.CrossEntropyLoss(reduction='mean')
    return lam * criterion(outputs, targets1) + (1 - lam) * criterion(outputs, targets2)

In [26]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [27]:
def rand_bbox_new(size, lam):
    H = size[2]
    W = size[3]

    x_margin_rate = 0.2

    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * (1-x_margin_rate*2) * cut_rat)
    cut_h = np.int(H * cut_rat)
    
    min_x_center = np.int(W * x_margin_rate + cut_w / 2)
    max_x_center = np.int(W * (1-x_margin_rate) - cut_w / 2)
    #print(min_x_center, max_x_center, lam, cut_w)
    min_y_center = cut_h // 2
    max_y_center = H - cut_h // 2
    if max_y_center == min_y_center:
        max_y_center += 1

    # uniform
    cx = np.random.randint(min_x_center, max_x_center)
    cy = np.random.randint(min_y_center, max_y_center)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    
    #print(bbx1, bbx2, bby1, bby2)

    return bbx1, bby1, bbx2, bby2

In [28]:
np.random.random()

0.4769354463088247

In [29]:
from over9000.over9000 import Over9000
from over9000.radam import RAdam
from gridmask import GridMask

In [30]:
from cvcore.solver import WarmupCyclicalLR
def make_optimizer(model, base_lr=4e-4, weight_decay=0., weight_decay_bias=0., epsilon=1e-3):
    """
    Create optimizer with per-layer learning rate and weight decay.
    """
    params = []
    for key, value in model.named_parameters():
        if not value.requires_grad:
            continue
        lr = base_lr
        params += [{"params": [value], "lr": lr, "weight_decay": weight_decay_bias if 'bias' in key else weight_decay}]
    
    optimizer = torch.optim.AdamW(params, lr, eps=epsilon)
    return optimizer

In [31]:
def train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid):
    train_loss = 0

    for batch_idx, (img, orig_img, targets) in enumerate(train_loader):
        img, orig_img, targets  = img.cuda(), orig_img.cuda(), targets.cuda()
        batch_size = img.size(0)
        r = np.random.rand()

        #if True:
        if r < 0.3:
            # generate mixed sample
            lam = np.random.beta(args.beta, args.beta)
            rand_index = torch.randperm(img.size()[0]).cuda()
            target_a = targets
            target_b = targets[rand_index]
            bbx1, bby1, bbx2, bby2 = rand_bbox(img.size(), lam)
            #img[:, :, bby1:bby2, bbx1:bbx2] = img[rand_index, :, bby1:bby2, bbx1:bbx2] #for new cutmix
            img[:, :, bbx1:bbx2, bby1:bby2] = img[rand_index, :, bbx1:bbx2, bby1:bby2]
            
            # adjust lambda to exactly match pixel ratio
            lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (img.size()[-1] * img.size()[-2]))
            # compute output
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, target_a) * lam + criterion(outputs, target_b) * (1. - lam)
            loss_aux1 = criterion(outputs_aux1, target_a) * lam + criterion(outputs_aux1, target_b) * (1. - lam)
            loss_aux2 = criterion(outputs_aux2, target_a) * lam + criterion(outputs_aux2, target_b) * (1. - lam)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        elif r > 0.7:
            img = grid(img)
            outputs, outputs_aux1, outputs_aux2 = model(img)
            loss_primary = criterion(outputs, targets)
            loss_aux1 = criterion(outputs_aux1, targets)
            loss_aux2 = criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
        else:
            orig_img, targets = mixup(orig_img, targets)
            outputs, outputs_aux1, outputs_aux2 = model(orig_img)
            loss_primary = mixup_criterion(outputs, targets)
            loss_aux1 = mixup_criterion(outputs_aux1, targets)
            loss_aux2 = mixup_criterion(outputs_aux2, targets)
            loss = loss_primary + (loss_aux1 + loss_aux2)*0.8
            #loss = criterion(outputs, targets)

        optimizer.zero_grad()
        with amp.scale_loss(loss, optimizer) as scaled_loss:
            scaled_loss.backward()
        #loss.backward()
        lr_scheduler(optimizer, batch_idx, epoch)
        optimizer.step()            
        
        current_lr = get_lrs(optimizer)

        train_loss += loss.item()
        print('\r {:4d} | {:.6f} | {:06d}/{} | {:.4f} | {:.4f} |'.format(
            epoch, float(current_lr[0]), batch_size*(batch_idx+1), train_loader.num, 
            loss.item(), train_loss/(batch_idx+1)), end='')


In [32]:
best_metrics = 0.

def validate_and_save(model, model_file, val_loader, save=False):
    global best_metrics
    best_key = 'recall'
    val_metrics = validate(model, val_loader)
    print('\nval:', val_metrics)
    
    if val_metrics[best_key] > best_metrics:
        best_metrics = val_metrics[best_key]
        if save:
            save_model(model, model_file)
            print('###>>>>> saved', model_file)
    model.train()
    

def train(args):
    model, model_file = create_model(cfg)
    model = model.cuda()

    swa_model, _ = create_model(cfg)
    swa_model = swa_model.cuda()
    swa_model_file = model_file

    optimizer = make_optimizer(model)
    lr_scheduler = WarmupCyclicalLR(
        "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)
    
    [model, swa_model], optimizer = amp.initialize([model, swa_model], optimizer, opt_level="O1",verbosity=0)
    
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
        swa_model = nn.DataParallel(swa_model)
    
    validate_and_save(model, model_file, val_loader, save=False)
    
    for cycle in range(1, args.num_cycles+1):
        print('CYCLE:', cycle)
        grid = GridMask(64, 128, rotate=15, ratio=0.6, mode=1, prob=1.)

        for epoch in range(args.num_epochs):
            grid.set_prob(epoch, args.st_epochs)
            train_epoch(args, model, train_loader, epoch, optimizer, lr_scheduler, grid)
            validate_and_save(model, model_file, val_loader, save=True)
            
            if (epoch+1) == args.swa_start and cycle == 1:
                copy_model(swa_model, model)
                swa_n = 0
            if (epoch+1) >= args.swa_start and (epoch+1) % args.swa_freq == 0:
                print('SWA>>>:')
                moving_average(swa_model, model, 1.0 / (swa_n + 1))
                swa_n += 1
                bn_update(train_loader, swa_model)
                validate_and_save(swa_model, swa_model_file, val_loader, save=True)

        #args.base_lr = 4e-4
        #args.num_epochs = 80
        #args.warmup_epochs = 100

        # reset scheduler at each cycle
        lr_scheduler = WarmupCyclicalLR(
            "cos", args.base_lr, args.num_epochs, iters_per_epoch=len(train_loader), warmup_epochs=args.warmup_epochs)

In [33]:
cfg = Namespace()
cfg.MODEL_NAME = 'tf_efficientnet_b4'
cfg.PRETRAINED = True
cfg.IN_CHANNELS = 1
cfg.POOL_TYPE = 'avg'
cfg.CLS_HEAD = 'linear'
cfg.MODEL_ACTIVATION = 'swish'
cfg.DROP_CONNECT = 0.2
cfg.DROPOUT= 0.
cfg.NUM_WORD_CLASSES = 1295
cfg.NUM_GRAPHEME_CLASSES = 168
cfg.NUM_VOWEL_CLASSES = 11
cfg.NUM_CONSONANT_CLASSES = 7
cfg.CKP_NAME = 'model4_eb4_fold3.pth'

In [34]:
#model, model_file = create_model(cfg)
#model(torch.randn(2,1,137,236))[0].size()

In [35]:
args = Namespace()

args.base_lr = 3e-4
args.num_epochs = 100
args.warmup_epochs = 5
args.num_cycles = 100
args.batch_size = 640
args.val_batch_size = 1024
args.st_epochs = 5

args.swa_start = 15
args.swa_freq = 3

args.beta = 1.0
args.cutmix_prob = 0.5

In [36]:
train_loader, val_loader = get_train_val_loaders(batch_size=args.batch_size, val_batch_size=args.val_batch_size, ifold=3)

(200840, 6)
(200840, 32332)
(160716, 6) (40124, 6)


In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...

val: {'recall': 0.997677, 'recall_grapheme': 0.996545, 'recall_vowel': 0.998518, 'recall_consonant': 0.999102, 'recall_word': 0.996328, 'acc_grapheme': 0.996212, 'acc_vowel': 0.99843, 'acc_consonant': 0.998928, 'acc_word': 0.996411, 'loss_grapheme': 0.04441, 'loss_vowel': 0.032399, 'loss_consonant': 0.022829, 'loss_word': 0.034176}
CYCLE: 1
    0 | 0.000060 | 160640/160716 | 12.7886 | 7.6774 |
val: {'recall': 0.997053, 'recall_grapheme': 0.995473, 'recall_vowel': 0.998275, 'recall_consonant': 0.998993, 'recall_word': 0.994994, 'acc_grapheme': 0.994667, 'acc_vowel': 0.998181, 'acc_consonant': 0.998554, 'acc_word': 0.994916, 'loss_grapheme': 0.06381, 'loss_vowel': 0.045205, 'loss_consonant': 0.029698, 'l

  0%|          | 0/251 [00:00<?, ?it/s]


val: {'recall': 0.997511, 'recall_grapheme': 0.996246, 'recall_vowel': 0.998392, 'recall_consonant': 0.999158, 'recall_word': 0.995686, 'acc_grapheme': 0.996012, 'acc_vowel': 0.998355, 'acc_consonant': 0.999003, 'acc_word': 0.995688, 'loss_grapheme': 0.031002, 'loss_vowel': 0.019486, 'loss_consonant': 0.013525, 'loss_word': 0.029986}
SWA>>>:


100%|██████████| 251/251 [02:00<00:00,  2.11it/s]



val: {'recall': 0.99768, 'recall_grapheme': 0.996871, 'recall_vowel': 0.998238, 'recall_consonant': 0.998742, 'recall_word': 0.996579, 'acc_grapheme': 0.996984, 'acc_vowel': 0.998355, 'acc_consonant': 0.999128, 'acc_word': 0.996611, 'loss_grapheme': 0.013851, 'loss_vowel': 0.007201, 'loss_consonant': 0.003333, 'loss_word': 0.014487}
   15 | 0.000281 | 160640/160716 | 5.6696 | 5.7706 ||
val: {'recall': 0.997285, 'recall_grapheme': 0.995867, 'recall_vowel': 0.998387, 'recall_consonant': 0.999019, 'recall_word': 0.995348, 'acc_grapheme': 0.995639, 'acc_vowel': 0.99843, 'acc_consonant': 0.998754, 'acc_word': 0.995339, 'loss_grapheme': 0.024124, 'loss_vowel': 0.013489, 'loss_consonant': 0.009732, 'loss_word': 0.02389}
   16 | 0.000279 | 160640/160716 | 2.9439 | 5.5401 ||
val: {'recall': 0.997826, 'recall_grapheme': 0.996754, 'recall_vowel': 0.998656, 'recall_consonant': 0.999141, 'recall_word': 0.996629, 'acc_grapheme': 0.996735, 'acc_vowel': 0.99843, 'acc_consonant': 0.998978, 'acc_word':

...

  0%|          | 0/251 [00:00<?, ?it/s]


val: {'recall': 0.997479, 'recall_grapheme': 0.996104, 'recall_vowel': 0.998606, 'recall_consonant': 0.999104, 'recall_word': 0.996002, 'acc_grapheme': 0.996386, 'acc_vowel': 0.99848, 'acc_consonant': 0.998953, 'acc_word': 0.996037, 'loss_grapheme': 0.020992, 'loss_vowel': 0.012042, 'loss_consonant': 0.007497, 'loss_word': 0.020735}
SWA>>>:


100%|██████████| 251/251 [02:01<00:00,  2.15it/s]



val: {'recall': 0.998264, 'recall_grapheme': 0.997393, 'recall_vowel': 0.998892, 'recall_consonant': 0.999377, 'recall_word': 0.997478, 'acc_grapheme': 0.997657, 'acc_vowel': 0.998829, 'acc_consonant': 0.999277, 'acc_word': 0.997483, 'loss_grapheme': 0.01106, 'loss_vowel': 0.005747, 'loss_consonant': 0.002705, 'loss_word': 0.011731}
   39 | 0.000196 | 160640/160716 | 0.2881 | 5.1186 |||
val: {'recall': 0.997828, 'recall_grapheme': 0.996578, 'recall_vowel': 0.999041, 'recall_consonant': 0.999116, 'recall_word': 0.996436, 'acc_grapheme': 0.99671, 'acc_vowel': 0.998554, 'acc_consonant': 0.999078, 'acc_word': 0.996486, 'loss_grapheme': 0.017612, 'loss_vowel': 0.009745, 'loss_consonant': 0.006062, 'loss_word': 0.017897}
   40 | 0.000196 | 012800/160716 | 1.6293 | 7.6365 |||

KeyboardInterrupt: 

In [37]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...

val: {'recall': 0.998314, 'recall_grapheme': 0.997479, 'recall_vowel': 0.998946, 'recall_consonant': 0.999349, 'recall_word': 0.997489, 'acc_grapheme': 0.997782, 'acc_vowel': 0.998779, 'acc_consonant': 0.999277, 'acc_word': 0.997508, 'loss_grapheme': 0.011308, 'loss_vowel': 0.005673, 'loss_consonant': 0.002604, 'loss_word': 0.011838}
CYCLE: 1
    0 | 0.000000 | 000640/160716 | 6.5520 | 6.5520 |

KeyboardInterrupt: 

In [39]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...

val: {'recall': 0.997677, 'recall_grapheme': 0.996545, 'recall_vowel': 0.998518, 'recall_consonant': 0.999102, 'recall_word': 0.996328, 'acc_grapheme': 0.996212, 'acc_vowel': 0.99843, 'acc_consonant': 0.998928, 'acc_word': 0.996411, 'loss_grapheme': 0.04441, 'loss_vowel': 0.032399, 'loss_consonant': 0.022829, 'loss_word': 0.034176}
CYCLE: 1
    0 | 0.000030 | 160640/160716 | 15.9834 | 7.3059 |
val: {'recall': 0.996764, 'recall_grapheme': 0.995136, 'recall_vowel': 0.997971, 'recall_consonant': 0.998812, 'recall_word': 0.994204, 'acc_grapheme': 0.994143, 'acc_vowel': 0.997782, 'acc_consonant': 0.998255, 'acc_word': 0.994118, 'loss_grapheme': 0.114387, 'loss_vowel': 0.090832, 'loss_consonant': 0.053885, '

  0%|          | 0/251 [00:00<?, ?it/s]


val: {'recall': 0.997627, 'recall_grapheme': 0.996461, 'recall_vowel': 0.998641, 'recall_consonant': 0.998945, 'recall_word': 0.995976, 'acc_grapheme': 0.996137, 'acc_vowel': 0.998505, 'acc_consonant': 0.998829, 'acc_word': 0.995987, 'loss_grapheme': 0.024857, 'loss_vowel': 0.014233, 'loss_consonant': 0.009961, 'loss_word': 0.023093}
SWA>>>:


100%|██████████| 251/251 [01:57<00:00,  2.13it/s]



val: {'recall': 0.997933, 'recall_grapheme': 0.99691, 'recall_vowel': 0.998685, 'recall_consonant': 0.999227, 'recall_word': 0.996874, 'acc_grapheme': 0.997059, 'acc_vowel': 0.998579, 'acc_consonant': 0.999227, 'acc_word': 0.996959, 'loss_grapheme': 0.013259, 'loss_vowel': 0.006767, 'loss_consonant': 0.003875, 'loss_word': 0.013435}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth
    5 | 0.000146 | 160640/160716 | 2.5631 | 5.4354 ||

...

  0%|          | 0/251 [00:00<?, ?it/s]


val: {'recall': 0.997446, 'recall_grapheme': 0.996123, 'recall_vowel': 0.998434, 'recall_consonant': 0.999106, 'recall_word': 0.995564, 'acc_grapheme': 0.995913, 'acc_vowel': 0.998305, 'acc_consonant': 0.998953, 'acc_word': 0.995564, 'loss_grapheme': 0.034811, 'loss_vowel': 0.021176, 'loss_consonant': 0.013914, 'loss_word': 0.033351}
SWA>>>:


100%|██████████| 251/251 [01:59<00:00,  2.18it/s]



val: {'recall': 0.998165, 'recall_grapheme': 0.997245, 'recall_vowel': 0.998805, 'recall_consonant': 0.999363, 'recall_word': 0.997242, 'acc_grapheme': 0.997483, 'acc_vowel': 0.998629, 'acc_consonant': 0.999327, 'acc_word': 0.997283, 'loss_grapheme': 0.012223, 'loss_vowel': 0.005743, 'loss_consonant': 0.003031, 'loss_word': 0.012465}
   23 | 0.000098 | 149120/160716 | 13.2362 | 4.8029 |

KeyboardInterrupt: 

In [ ]:
train(args)

model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...
model file: ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth, exist: True
loading ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth...

val: {'recall': 0.996911, 'recall_grapheme': 0.995499, 'recall_vowel': 0.997956, 'recall_consonant': 0.99869, 'recall_word': 0.994599, 'acc_grapheme': 0.995414, 'acc_vowel': 0.997906, 'acc_consonant': 0.998604, 'acc_word': 0.994617, 'loss_grapheme': 0.023309, 'loss_vowel': 0.012818, 'loss_consonant': 0.008384, 'loss_word': 0.025105}
CYCLE: 1
    0 | 0.000040 | 160640/160716 | 2.8293 | 11.1408 ||
val: {'recall': 0.996439, 'recall_grapheme': 0.994609, 'recall_vowel': 0.997744, 'recall_consonant': 0.998792, 'recall_word': 0.993961, 'acc_grapheme': 0.994592, 'acc_vowel': 0.997807, 'acc_consonant': 0.99833, 'acc_word': 0.993969, 'loss_grapheme': 0.088315, 'loss_vowel': 0.069688, 'loss_consonant': 0.047815, 

   21 | 0.000354 | 160640/160716 | 12.0534 | 11.4987 |
val: {'recall': 0.995565, 'recall_grapheme': 0.99352, 'recall_vowel': 0.997462, 'recall_consonant': 0.997757, 'recall_word': 0.992915, 'acc_grapheme': 0.993171, 'acc_vowel': 0.997483, 'acc_consonant': 0.997707, 'acc_word': 0.992922, 'loss_grapheme': 0.105962, 'loss_vowel': 0.080502, 'loss_consonant': 0.057482, 'loss_word': 0.076235}
   22 | 0.000350 | 160640/160716 | 16.5776 | 11.6344 |
val: {'recall': 0.995819, 'recall_grapheme': 0.993562, 'recall_vowel': 0.997638, 'recall_consonant': 0.998515, 'recall_word': 0.992877, 'acc_grapheme': 0.993346, 'acc_vowel': 0.997508, 'acc_consonant': 0.997857, 'acc_word': 0.992872, 'loss_grapheme': 0.089941, 'loss_vowel': 0.069193, 'loss_consonant': 0.046205, 'loss_word': 0.072884}
   23 | 0.000346 | 160640/160716 | 6.9081 | 11.4663 ||
val: {'recall': 0.995978, 'recall_grapheme': 0.993998, 'recall_vowel': 0.997892, 'recall_consonant': 0.998024, 'recall_word': 0.993412, 'acc_grapheme': 0.99347, 'ac

   63 | 0.000115 | 160640/160716 | 13.9547 | 10.9493 |
val: {'recall': 0.996213, 'recall_grapheme': 0.994363, 'recall_vowel': 0.997496, 'recall_consonant': 0.998628, 'recall_word': 0.993742, 'acc_grapheme': 0.994168, 'acc_vowel': 0.997707, 'acc_consonant': 0.998405, 'acc_word': 0.993769, 'loss_grapheme': 0.06019, 'loss_vowel': 0.046974, 'loss_consonant': 0.034497, 'loss_word': 0.046325}
   64 | 0.000109 | 160640/160716 | 13.1597 | 11.4104 |
val: {'recall': 0.996515, 'recall_grapheme': 0.994747, 'recall_vowel': 0.997752, 'recall_consonant': 0.998813, 'recall_word': 0.994123, 'acc_grapheme': 0.994392, 'acc_vowel': 0.997857, 'acc_consonant': 0.99843, 'acc_word': 0.994143, 'loss_grapheme': 0.078374, 'loss_vowel': 0.062671, 'loss_consonant': 0.043452, 'loss_word': 0.057362}
   65 | 0.000104 | 160640/160716 | 3.4650 | 10.8686 ||
val: {'recall': 0.996517, 'recall_grapheme': 0.994764, 'recall_vowel': 0.997649, 'recall_consonant': 0.99889, 'recall_word': 0.99408, 'acc_grapheme': 0.994492, 'acc_

    5 | 0.000238 | 160640/160716 | 2.7851 | 10.3432 ||
val: {'recall': 0.996168, 'recall_grapheme': 0.994256, 'recall_vowel': 0.997589, 'recall_consonant': 0.99857, 'recall_word': 0.994029, 'acc_grapheme': 0.994268, 'acc_vowel': 0.997757, 'acc_consonant': 0.99838, 'acc_word': 0.994019, 'loss_grapheme': 0.081598, 'loss_vowel': 0.062646, 'loss_consonant': 0.042767, 'loss_word': 0.063334}
    6 | 0.000276 | 160640/160716 | 15.0636 | 11.1700 |
val: {'recall': 0.995897, 'recall_grapheme': 0.993542, 'recall_vowel': 0.997928, 'recall_consonant': 0.998576, 'recall_word': 0.99286, 'acc_grapheme': 0.993395, 'acc_vowel': 0.997757, 'acc_consonant': 0.998255, 'acc_word': 0.992922, 'loss_grapheme': 0.050508, 'loss_vowel': 0.035042, 'loss_consonant': 0.02805, 'loss_word': 0.041885}
    7 | 0.000315 | 160640/160716 | 11.8219 | 10.9983 |
val: {'recall': 0.995826, 'recall_grapheme': 0.993751, 'recall_vowel': 0.997616, 'recall_consonant': 0.998185, 'recall_word': 0.9934, 'acc_grapheme': 0.993371, 'acc_vo

   47 | 0.000213 | 160640/160716 | 10.9977 | 11.1353 |
val: {'recall': 0.996258, 'recall_grapheme': 0.994159, 'recall_vowel': 0.99806, 'recall_consonant': 0.998655, 'recall_word': 0.99421, 'acc_grapheme': 0.993969, 'acc_vowel': 0.998106, 'acc_consonant': 0.99838, 'acc_word': 0.994243, 'loss_grapheme': 0.089665, 'loss_vowel': 0.068297, 'loss_consonant': 0.046455, 'loss_word': 0.056737}
   48 | 0.000206 | 160640/160716 | 16.1691 | 10.9213 |
val: {'recall': 0.996482, 'recall_grapheme': 0.994521, 'recall_vowel': 0.998065, 'recall_consonant': 0.998819, 'recall_word': 0.994565, 'acc_grapheme': 0.994467, 'acc_vowel': 0.998056, 'acc_consonant': 0.998629, 'acc_word': 0.994517, 'loss_grapheme': 0.049576, 'loss_vowel': 0.036349, 'loss_consonant': 0.0285, 'loss_word': 0.038976}
   49 | 0.000200 | 160640/160716 | 13.5596 | 11.0215 |
val: {'recall': 0.996105, 'recall_grapheme': 0.993989, 'recall_vowel': 0.99792, 'recall_consonant': 0.998524, 'recall_word': 0.994313, 'acc_grapheme': 0.994168, 'acc_vo

...

   10 | 0.000388 | 160640/160716 | 0.9676 | 11.1211 ||
val: {'recall': 0.996934, 'recall_grapheme': 0.995485, 'recall_vowel': 0.998156, 'recall_consonant': 0.998609, 'recall_word': 0.995019, 'acc_grapheme': 0.995165, 'acc_vowel': 0.997981, 'acc_consonant': 0.99848, 'acc_word': 0.995015, 'loss_grapheme': 0.054211, 'loss_vowel': 0.039601, 'loss_consonant': 0.028926, 'loss_word': 0.046019}
   11 | 0.000386 | 160640/160716 | 14.7624 | 11.2504 |
val: {'recall': 0.996274, 'recall_grapheme': 0.994298, 'recall_vowel': 0.99792, 'recall_consonant': 0.998579, 'recall_word': 0.994121, 'acc_grapheme': 0.994218, 'acc_vowel': 0.997732, 'acc_consonant': 0.998255, 'acc_word': 0.994193, 'loss_grapheme': 0.070826, 'loss_vowel': 0.053538, 'loss_consonant': 0.039012, 'loss_word': 0.050347}
   12 | 0.000384 | 160640/160716 | 11.2385 | 10.8819 |
val: {'recall': 0.996868, 'recall_grapheme': 0.995271, 'recall_vowel': 0.998247, 'recall_consonant': 0.998682, 'recall_word': 0.994827, 'acc_grapheme': 0.995265, 'ac

   52 | 0.000181 | 160640/160716 | 13.1303 | 10.5975 |
val: {'recall': 0.99726, 'recall_grapheme': 0.995985, 'recall_vowel': 0.998387, 'recall_consonant': 0.998683, 'recall_word': 0.995658, 'acc_grapheme': 0.995788, 'acc_vowel': 0.99823, 'acc_consonant': 0.998704, 'acc_word': 0.995688, 'loss_grapheme': 0.036017, 'loss_vowel': 0.024975, 'loss_consonant': 0.019779, 'loss_word': 0.029314}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth
   53 | 0.000175 | 160640/160716 | 1.7507 | 10.5695 ||
val: {'recall': 0.997249, 'recall_grapheme': 0.995849, 'recall_vowel': 0.998636, 'recall_consonant': 0.998664, 'recall_word': 0.995473, 'acc_grapheme': 0.995614, 'acc_vowel': 0.998405, 'acc_consonant': 0.998629, 'acc_word': 0.995489, 'loss_grapheme': 0.044376, 'loss_vowel': 0.03024, 'loss_consonant': 0.022461, 'loss_word': 0.032081}
   54 | 0.000169 | 160640/160716 | 3.6021 | 10.4090 ||
val: {'recall': 0.997623, 'recall_grapheme': 0.996501, 'recall_vowel': 0.998383, 'recall_consonan

   74 | 0.000059 | 160640/160716 | 13.0889 | 10.3437 |
val: {'recall': 0.997026, 'recall_grapheme': 0.995506, 'recall_vowel': 0.998171, 'recall_consonant': 0.998919, 'recall_word': 0.995206, 'acc_grapheme': 0.99519, 'acc_vowel': 0.998156, 'acc_consonant': 0.998554, 'acc_word': 0.99524, 'loss_grapheme': 0.04657, 'loss_vowel': 0.032226, 'loss_consonant': 0.022714, 'loss_word': 0.030321}
   75 | 0.000054 | 160640/160716 | 10.4996 | 10.1636 |
val: {'recall': 0.997516, 'recall_grapheme': 0.996264, 'recall_vowel': 0.998418, 'recall_consonant': 0.999119, 'recall_word': 0.995943, 'acc_grapheme': 0.996237, 'acc_vowel': 0.99833, 'acc_consonant': 0.998878, 'acc_word': 0.995987, 'loss_grapheme': 0.028782, 'loss_vowel': 0.017658, 'loss_consonant': 0.013478, 'loss_word': 0.025172}
   76 | 0.000050 | 160640/160716 | 13.7797 | 9.9986 ||
val: {'recall': 0.997508, 'recall_grapheme': 0.996182, 'recall_vowel': 0.998495, 'recall_consonant': 0.999172, 'recall_word': 0.996009, 'acc_grapheme': 0.996037, 'acc_

   16 | 0.000372 | 160640/160716 | 9.2786 | 10.8445 ||
val: {'recall': 0.996903, 'recall_grapheme': 0.995879, 'recall_vowel': 0.998321, 'recall_consonant': 0.997533, 'recall_word': 0.995351, 'acc_grapheme': 0.995539, 'acc_vowel': 0.998305, 'acc_consonant': 0.998554, 'acc_word': 0.995364, 'loss_grapheme': 0.037043, 'loss_vowel': 0.02585, 'loss_consonant': 0.021044, 'loss_word': 0.030192}
   17 | 0.000369 | 160640/160716 | 15.2112 | 10.5779 |
val: {'recall': 0.996994, 'recall_grapheme': 0.995431, 'recall_vowel': 0.998308, 'recall_consonant': 0.998805, 'recall_word': 0.99513, 'acc_grapheme': 0.99519, 'acc_vowel': 0.998405, 'acc_consonant': 0.99843, 'acc_word': 0.995065, 'loss_grapheme': 0.055552, 'loss_vowel': 0.041279, 'loss_consonant': 0.030041, 'loss_word': 0.037895}
   18 | 0.000365 | 160640/160716 | 11.4803 | 10.6635 |
val: {'recall': 0.997014, 'recall_grapheme': 0.995216, 'recall_vowel': 0.998602, 'recall_consonant': 0.999022, 'recall_word': 0.995238, 'acc_grapheme': 0.995215, 'acc_

   58 | 0.000144 | 160640/160716 | 10.6053 | 10.6897 |
val: {'recall': 0.997544, 'recall_grapheme': 0.996517, 'recall_vowel': 0.998492, 'recall_consonant': 0.998649, 'recall_word': 0.996338, 'acc_grapheme': 0.996361, 'acc_vowel': 0.99853, 'acc_consonant': 0.998854, 'acc_word': 0.996361, 'loss_grapheme': 0.032809, 'loss_vowel': 0.021978, 'loss_consonant': 0.015902, 'loss_word': 0.02601}
   59 | 0.000138 | 160640/160716 | 10.3413 | 10.4644 |
val: {'recall': 0.997714, 'recall_grapheme': 0.996584, 'recall_vowel': 0.998596, 'recall_consonant': 0.999091, 'recall_word': 0.996396, 'acc_grapheme': 0.996386, 'acc_vowel': 0.998505, 'acc_consonant': 0.998854, 'acc_word': 0.996436, 'loss_grapheme': 0.035497, 'loss_vowel': 0.02408, 'loss_consonant': 0.018238, 'loss_word': 0.026557}
###>>>>> saved ./model4-ckps/tf_efficientnet_b4/model4_eb4_fold3.pth
   60 | 0.000132 | 160640/160716 | 13.2141 | 10.2445 |

In [ ]:
#save_model(model, model_file)

In [ ]:
del model